In [1]:
import pandas as pd
from numpy import *
import numpy as np
from scipy import sparse



data_playlists= pd.read_csv('all/train.csv',low_memory = False)
data_tracks= pd.read_csv('all/tracks.csv',low_memory = False)



#SPLITTAGGIO DEI DATI

#prendo il 20% delle random playlist da target playlist
target_playlist= pd.read_csv('all/target_playlists.csv',low_memory = False)
random_playlist=target_playlist.iloc[5000:10000]
random_id = random_playlist.playlist_id.values
random_data=data_playlists.loc[data_playlists['playlist_id'].isin(random_id)]
#random_data.shape = (114513, 2)
n=round((114513/100)*20)
random_subset = random_data.sample(n=n)

#prendo il 20% delle sequential playlist da target playlist
sequential_playlist=target_playlist.iloc[:5000]
sequential_id = sequential_playlist.playlist_id.values
sequential_data=data_playlists.loc[data_playlists['playlist_id'].isin(sequential_id)]
n1=round((115553/100)*20)
for pid in sequential_id:
    playlistID=sequential_data.loc[sequential_data['playlist_id']==pid]
    start=sequential_data.index[sequential_data['playlist_id'] == pid][0]
    end=start+round((playlistID.shape[0]*50)/100)

    dropping=list(range(start, end))    
    sequential_data=sequential_data.drop(dropping)


sequential_subset = sequential_data.sample(n=n1)

#SETTO URM TEST
URM_test=zeros((50446,20635))
#20% from random
for row in random_subset.itertuples():
    riga = row.playlist_id
    #print(riga)
    colonna= row.track_id
    #print(colonna)
    URM_test[riga,colonna]=1
#20% from random
for row in sequential_subset.itertuples():
    riga = row.playlist_id
    #print(riga)
    colonna= row.track_id
    #print(colonna)
    URM_test[riga,colonna]=1

#SETTO URM TRAIN e ALL
URM_train=zeros((50446,20635))
URM_all=zeros((50446,20635))
#inizializzazione completa
for row in data_playlists.itertuples():
    riga = row.playlist_id
    #print(riga)
    colonna= row.track_id
    #print(colonna)
    URM_all[riga,colonna]=1
    URM_train[riga,colonna]=1
#rimozione del 20%from random
for row in random_subset.itertuples():
    riga = row.playlist_id
    #print(riga)
    colonna= row.track_id
    #print(colonna)
    URM_train[riga,colonna]=0
#rimozione del 20%from sequential
for row in sequential_subset.itertuples():
    riga = row.playlist_id
    #print(riga)
    colonna= row.track_id
    #print(colonna)
    URM_train[riga,colonna]=0


#CREAZIONE ICM
#Creare la icm con righe: tracks , colonne:features
#Moltipolicare icm per icmT ottenendo la matrice delle similarità
#Rimuovere la diagonale dalla matrice S (funzione lil..)
#Moltiplicare la matrice delle playlist (URM)) per quella delle S ottenendo una votazione
ICM_all=zeros((20635,19412))

#inizializzazione album
for row in data_tracks.itertuples():
    riga = row.track_id
    colonna= row.album_id
    ICM_all[riga,colonna]=1
#test, deve tornare 1   
print (ICM_all[3,3989])

#inizializzazione artist
for row in data_tracks.itertuples():
    riga = row.track_id
    colonna= row.artist_id
    ICM_all[riga,12744+colonna]=0.5
#test, deve tornare 1   
print (ICM_all[1,17647])

'''
#CREAZIONE DELLA UCM_ALL
UCM_all=zeros((50446,19412))


#inizializzazione completa
for row in data_playlists.itertuples():
    riga = row.playlist_id
    #print(riga)
    track=data_tracks.loc[data_tracks['track_id'] == row.track_id]
    #print(colonna)
    UCM_all[riga,track.album_id]=1
    UCM_all[riga,12744+track.artist_id]=1
#rimozione del 20%from random
for row in random_subset.itertuples():
    riga = row.playlist_id
    #print(riga)
    track=data_tracks.loc[data_tracks['track_id'] == row.track_id]
    #print(colonna)
    UCM_all[riga,track.album_id]=0
    UCM_all[riga,12744+track.artist_id]=0
#rimozione del 20%from sequential
for row in sequential_subset.itertuples():
    riga = row.playlist_id
    #print(riga)
    track=data_tracks.loc[data_tracks['track_id'] == row.track_id]
    #print(colonna)
    UCM_all[riga,track.album_id]=0
    UCM_all[riga,12744+track.artist_id]=0
    
    UCM_train=sparse.csr_matrix(UCM_all)   
    sparse.save_npz('UCM_train_matrix.npz', UCM_train)

'''

In [2]:
URM_all=sparse.csr_matrix(URM_all)   
URM_train=sparse.csr_matrix(URM_train)
URM_test=sparse.csr_matrix(URM_test) 
ICM_all=sparse.csr_matrix(ICM_all) 

In [6]:
sparse.save_npz('URM_all_matrix.npz', URM_all)
sparse.save_npz('URM_train_matrix.npz', URM_train)
sparse.save_npz('URM_test_matrix.npz', URM_test)
sparse.save_npz('ICM_all_matrix.npz', ICM_all)

In [ ]:
#CREAZIONE DELLE MATRICI PER LA SOLA PARTE RANDOM E LA SOLA PARTE SEQUENTIAL
import traceback, os
import scipy.sparse
import pandas as pd
import numpy as np

URM_all = scipy.sparse.load_npz('./Matrix/URM_all_matrix.npz')
ICM_all = scipy.sparse.load_npz('./Matrix/ICM_all_matrix.npz')
URM_train = scipy.sparse.load_npz('./Matrix/URM_train_matrix.npz')
URM_test = scipy.sparse.load_npz('./Matrix/URM_test_matrix.npz')
UCM_all = scipy.sparse.load_npz('./Matrix/UCM_all_matrix.npz')


#Load playlistsIDS
train_df= pd.read_csv('all/train.csv',low_memory = False)
all_userID=train_df.playlist_id.unique()

target_playlist_df=pd.read_csv('all/target_playlists.csv',low_memory = False)
target_userID=target_playlist_df.playlist_id.unique()

sequential_playlist=target_playlist_df.iloc[:5000]
sequential_id = sequential_playlist.playlist_id.values.tolist()
sequential_df=train_df.loc[train_df['playlist_id'].isin(sequential_id)]

#Selezione il 20% da inserire in URM_test e lo metto in random_subset
random_playlist=target_playlist_df.iloc[5000:10000]
random_id = random_playlist.playlist_id.values.tolist()
random_data_df=train_df.loc[train_df['playlist_id'].isin(random_id)]
#random_data.shape = (114513, 2)
n=round((114513/100)*20)
random_subset = random_data_df.sample(n=n)

#Selezione il 20% da inserire in URM_test e lo metto in sequential_subset
sequential_data=train_df.loc[train_df['playlist_id'].isin(sequential_id)]
n1=round((115553/100)*20)
for pid in sequential_id:
    playlistID=sequential_data.loc[sequential_data['playlist_id']==pid]
    start=sequential_data.index[sequential_data['playlist_id'] == pid][0]
    end=start+round((playlistID.shape[0]*50)/100)

    dropping=list(range(start, end))    
    sequential_data=sequential_data.drop(dropping)


sequential_subset = sequential_data.sample(n=n1)


from numpy import *
    
#--------------SEQUENTIAL---------------------------    
URM_train_sequential=zeros((5000,20635))
URM_test_sequential=zeros((5000,20635))

for row in sequential_subset.itertuples():
    riga = sequential_id.index(row.playlist_id)
    #print(riga)
    colonna= row.track_id
    #print(colonna)
    URM_test_sequential[riga,colonna]=1

for row in sequential_df.itertuples():
    #print(riga)
    riga= sequential_id.index(row.playlist_id)
    colonna= row.track_id
    #print(colonna)
    URM_train_sequential[riga,colonna]=1
    
for row in sequential_subset.itertuples():
    riga = sequential_id.index(row.playlist_id)
    #print(riga)
    colonna= row.track_id
    #print(colonna)
    URM_train_sequential[riga,colonna]=0
    
    
#--------------RANDOM---------------------------    
URM_train_random=zeros((5000,20635))
URM_train_random=zeros((5000,20635))

for row in random_subset.itertuples():
    riga = random_id.index(row.playlist_id)
    #print(riga)
    colonna= row.track_id
    #print(colonna)
    URM_train_random[riga,colonna]=1

for row in random_data_df.itertuples():
    #print(riga)
    riga= random_id.index(row.playlist_id)
    colonna= row.track_id
    #print(colonna)
    URM_train_random[riga,colonna]=1
    
for row in random_subset.itertuples():
    riga = random_id.index(row.playlist_id)
    #print(riga)
    colonna= row.track_id
    #print(colonna)
    URM_train_random[riga,colonna]=0
    
    
from scipy import sparse

URM_train_sequential=sparse.csr_matrix(URM_train_sequential)
URM_test_sequential=sparse.csr_matrix(URM_test_sequential) 
sparse.save_npz('URM_train_sequential.npz', sequential_URM_train)
sparse.save_npz('URM_test_sequential.npz', sequential_URM_test)

URM_train_random=sparse.csr_matrix(URM_train_random)
URM_test_random=sparse.csr_matrix(URM_test_random) 
sparse.save_npz('URM_train_random.npz', URM_train_random)
sparse.save_npz('URM_test_random.npz', URM_test_random)


